## Deploy an MLflow model with SageMaker

In [ ]:
!pip install -q mlflow==1.12.1

In [ ]:
import json
import boto3
import mlflow
import sagemaker
import pandas as pd
import mlflow.sagemaker
from sklearn.datasets import load_boston

In [ ]:
region = sagemaker.Session().boto_region_name
role = sagemaker.get_execution_role()
tracking_uri = '<MLFLOW Server URI>'
mlflow.set_tracking_uri(tracking_uri)

In [ ]:
!mlflow sagemaker build-and-push-container

In [ ]:
image_uri = '<YOUR mlflow-pyfunc ECR IMAGE URI>'

# Deploy a sageMaker endpoint with our scikit-learn model

In [ ]:
endpoint_name = 'boston-housing'
model_uri = 'models:/<model_name>/<model_version>'

In [ ]:
mlflow.sagemaker.deploy(
    mode ='create',
    app_name=endpoint_name,
    model_uri=model_uri,
    image_uri=image_uri,
    execution_role_arn=role,
    instance_type='ml.m5.xlarge',
    instance_count=1,
    region_name=region
)

In [ ]:
data = load_boston()
df = pd.DataFrame(data.data, columns=data.feature_names)

In [ ]:
runtime = boto3.client('runtime.sagemaker')
payload = df.iloc[[0]].to_json(orient="split")

runtime_response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=payload
)

result = json.loads(runtime_response['Body'].read().decode())
print(f'Payload: {payload}')
print(f'Prediction: {result}')

In [ ]:
mlflow.sagemaker.delete(app_name=endpoint_name, region_name=region)